In [1]:
# Import dependencies
import pandas as pd 
import sqlite3

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

## Cleaning County-Based Data 

In [45]:
# Import csv file 
county_file = "NCHS_-_Teen_Birth_Rates_for_Age_Group_15-19_in_the_United_States_by_County.csv"
county_df = pd.read_csv(county_file)
county_df

,Year,State,County,State FIPS Code,County FIPS Code,Combined FIPS Code,Birth Rate,Lower Confidence Limit,Upper Confidence Limit
0,2003,Alabama,Autauga,1,1,1001,48.4,42.1,55.2
1,2004,Alabama,Autauga,1,1,1001,46.4,41.6,51.4
2,2005,Alabama,Autauga,1,1,1001,44.1,40.3,48.1
3,2006,Alabama,Autauga,1,1,1001,44.3,40.8,48.0
4,2007,Alabama,Autauga,1,1,1001,43.7,40.3,47.3
...,...,...,...,...,...,...,...,...,...
50187,2014,Wyoming,Weston,56,45,56045,30.4,22.6,39.1
50188,2015,Wyoming,Weston,56,45,56045,28.6,20.6,37.9
50189,2016,Wyoming,Weston,56,45,56045,26.8,18.4,36.7
50190,2017,Wyoming,Weston,56,45,56045,25.4,16.6,36.3


In [46]:
# Renaming columns to remove spaces
county_df.rename(columns={'Year': 'year', 'State':'state', 'County':'county', 'State FIPS Code':'state_fips_code', 
                        'County FIPS Code':'county_fips_code','Combined FIPS Code': 'combined_fips_code', 
                        'Birth Rate':'birth_rate', 'Lower Confidence Limit':'lower_confidence_limit',
                        'Upper Confidence Limit':'upper_confidence_limit'}, inplace=True)

county_df.head(10)

,year,state,county,state_fips_code,county_fips_code,combined_fips_code,birth_rate,lower_confidence_limit,upper_confidence_limit
0,2003,Alabama,Autauga,1,1,1001,48.4,42.1,55.2
1,2004,Alabama,Autauga,1,1,1001,46.4,41.6,51.4
2,2005,Alabama,Autauga,1,1,1001,44.1,40.3,48.1
3,2006,Alabama,Autauga,1,1,1001,44.3,40.8,48.0
4,2007,Alabama,Autauga,1,1,1001,43.7,40.3,47.3
5,2008,Alabama,Autauga,1,1,1001,41.7,38.4,45.1
6,2009,Alabama,Autauga,1,1,1001,38.8,35.6,42.0
7,2010,Alabama,Autauga,1,1,1001,34.8,31.9,37.8
8,2011,Alabama,Autauga,1,1,1001,32.0,29.2,34.9
9,2012,Alabama,Autauga,1,1,1001,30.4,27.7,33.2


In [47]:
# Separate out state codes to prep to add in leading zeroes 
county_df_lessthan10 = county_df[county_df['state_fips_code'] < 10]
county_df_greaterthan10 = county_df[county_df['state_fips_code'] >= 10]

# Change state codes to string types
county_df_greaterthan10['state_fips_code'] = county_df_greaterthan10['state_fips_code'].astype('str')
# Changing to string and adding a leading zero to state codes less than 10 (e.g., 01, 05, etc.) 
county_df_lessthan10['state_fips_code'] = county_df_lessthan10['state_fips_code'].astype('str').str.zfill(2)

# Concat the rows together back into a full df
county_df = pd.concat([county_df_lessthan10, county_df_greaterthan10])
county_df


<ipython-input-47-36788996cbae>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df_greaterthan10['state_fips_code'] = county_df_greaterthan10['state_fips_code'].astype('str')
<ipython-input-47-36788996cbae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df_lessthan10['state_fips_code'] = county_df_lessthan10['state_fips_code'].astype('str').str.zfill(2)


,year,state,county,state_fips_code,county_fips_code,combined_fips_code,birth_rate,lower_confidence_limit,upper_confidence_limit
0,2003,Alabama,Autauga,01,1,1001,48.4,42.1,55.2
1,2004,Alabama,Autauga,01,1,1001,46.4,41.6,51.4
2,2005,Alabama,Autauga,01,1,1001,44.1,40.3,48.1
3,2006,Alabama,Autauga,01,1,1001,44.3,40.8,48.0
4,2007,Alabama,Autauga,01,1,1001,43.7,40.3,47.3
...,...,...,...,...,...,...,...,...,...
50187,2014,Wyoming,Weston,56,45,56045,30.4,22.6,39.1
50188,2015,Wyoming,Weston,56,45,56045,28.6,20.6,37.9
50189,2016,Wyoming,Weston,56,45,56045,26.8,18.4,36.7
50190,2017,Wyoming,Weston,56,45,56045,25.4,16.6,36.3


In [48]:
# convert county fips & combined fips codes to string types 
county_df['county_fips_code'] = county_df['county_fips_code'].astype(str)
county_df['combined_fips_code'] = county_df['combined_fips_code'].astype(str)
county_df

,year,state,county,state_fips_code,county_fips_code,combined_fips_code,birth_rate,lower_confidence_limit,upper_confidence_limit
0,2003,Alabama,Autauga,01,1,1001,48.4,42.1,55.2
1,2004,Alabama,Autauga,01,1,1001,46.4,41.6,51.4
2,2005,Alabama,Autauga,01,1,1001,44.1,40.3,48.1
3,2006,Alabama,Autauga,01,1,1001,44.3,40.8,48.0
4,2007,Alabama,Autauga,01,1,1001,43.7,40.3,47.3
...,...,...,...,...,...,...,...,...,...
50187,2014,Wyoming,Weston,56,45,56045,30.4,22.6,39.1
50188,2015,Wyoming,Weston,56,45,56045,28.6,20.6,37.9
50189,2016,Wyoming,Weston,56,45,56045,26.8,18.4,36.7
50190,2017,Wyoming,Weston,56,45,56045,25.4,16.6,36.3


In [49]:
county_df.dtypes

year                        int64
state                      object
county                     object
state_fips_code            object
county_fips_code           object
combined_fips_code         object
birth_rate                float64
lower_confidence_limit    float64
upper_confidence_limit    float64
dtype: object

## Cleaning State-Based Data

In [50]:
# Import second csv file for national and total state birth rate comparison
national_file = "NCHS_-_U.S._and_State_Trends_on_Teen_Births.csv"
national_df = pd.read_csv(national_file)
national_df

,Year,State,Age Group (Years),State Rate,State Births,U.S. Births,U.S. Birth Rate,Unit
0,1990,Alabama,15-17 years,47.4,4222,183327,37.5,"per 1,000"
1,1990,Alaska,15-17 years,31.2,335,183327,37.5,"per 1,000"
2,1990,Arizona,15-17 years,47.7,3436,183327,37.5,"per 1,000"
3,1990,Arkansas,15-17 years,50.4,2549,183327,37.5,"per 1,000"
4,1990,California,15-17 years,44.6,24880,183327,37.5,"per 1,000"
...,...,...,...,...,...,...,...,...
4519,2018,Virginia,18-19 years,26.2,2918,135580,32.3,"per 1,000"
4520,2018,Washington,18-19 years,25.0,2128,135580,32.3,"per 1,000"
4521,2018,West Virginia,18-19 years,50.4,1057,135580,32.3,"per 1,000"
4522,2018,Wisconsin,18-19 years,24.4,1865,135580,32.3,"per 1,000"


In [51]:
# Rename column names to remove spaces
national_df.rename(columns={'Year': 'year', 'State': 'state', 'Age Group (Years)': 'age_group', 'State Births': 'state_births',
                            'State Rate': 'state_rate', 'U.S. Births': 'us_births', 'U.S. Birth Rate': 'us_rate'}, inplace=True)
# Drop unnecessary "Unit" column (every value = "per 1,000")
national_df = national_df[['year', 'state', 'age_group', 'state_births', 'state_rate', 'us_births', 'us_rate']]
national_df

,year,state,age_group,state_births,state_rate,us_births,us_rate
0,1990,Alabama,15-17 years,4222,47.4,183327,37.5
1,1990,Alaska,15-17 years,335,31.2,183327,37.5
2,1990,Arizona,15-17 years,3436,47.7,183327,37.5
3,1990,Arkansas,15-17 years,2549,50.4,183327,37.5
4,1990,California,15-17 years,24880,44.6,183327,37.5
...,...,...,...,...,...,...,...
4519,2018,Virginia,18-19 years,2918,26.2,135580,32.3
4520,2018,Washington,18-19 years,2128,25.0,135580,32.3
4521,2018,West Virginia,18-19 years,1057,50.4,135580,32.3
4522,2018,Wisconsin,18-19 years,1865,24.4,135580,32.3


In [52]:
# Filter years to 2003-2018
national_df = national_df[national_df['year'] >= 2003]
national_df

,year,state,age_group,state_births,state_rate,us_births,us_rate
676,2003,Alabama,15-17 years,2665,28.5,134384,22.2
677,2003,Alaska,15-17 years,320,19.8,134384,22.2
678,2003,Arizona,15-17 years,4133,36.2,134384,22.2
679,2003,Arkansas,15-17 years,1728,30.4,134384,22.2
680,2003,California,15-17 years,16197,21.5,134384,22.2
...,...,...,...,...,...,...,...
4519,2018,Virginia,18-19 years,2918,26.2,135580,32.3
4520,2018,Washington,18-19 years,2128,25.0,135580,32.3
4521,2018,West Virginia,18-19 years,1057,50.4,135580,32.3
4522,2018,Wisconsin,18-19 years,1865,24.4,135580,32.3


## Exporting to SQLite

In [53]:
# Export dfs to SQLite
county_df.to_sql(name='county', con=engine)
national_df.to_sql(name='national', con=engine)


### Double-Checking Classes and Work

In [2]:
# Checking classes were made successfully: https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

In [3]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [4]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [5]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['county', 'national']

In [6]:
# Assign the classes to variables
County = Base.classes.county
National = Base.classes.national

In [7]:
# Create a session
session = Session(engine)

In [64]:
# State-level: Display the row's columns and data in dictionary format
first_row_national = session.query(National).first()
first_row_national.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7ff36aadfe80>,
 'us_births': 134384,
 'state_rate': 28.5,
 'age_group': '15-17 years',
 'year': 2003,
 'us_rate': 22.2,
 'state_births': 2665,
 'state': 'Alabama',
 'index': 676}

In [65]:
# County-level: Display the row's columns and data in dictionary format 
first_row_county = session.query(County).first()
first_row_county.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7ff36e484820>,
 'state_fips_code': '01',
 'state': 'Alabama',
 'index': 0,
 'upper_confidence_limit': 55.2,
 'birth_rate': 48.4,
 'county_fips_code': '1',
 'county': 'Autauga',
 'year': 2003,
 'lower_confidence_limit': 42.1,
 'combined_fips_code': '1001'}

In [87]:
session.close()
